# CONV NETS Testing

In [5]:
import numpy as np
import h5py
from utils_cnn.utils import load_dataset
from utils_cnn.utils import convert_to_one_hot
from utils_cnn.utils import linear_forward
from utils_cnn.forward import conv_layer
from utils_cnn.forward import pool_forward
from utils_cnn.backward import convolutional_backward
from utils_cnn.backward import pool_backward
from utils_cnn.activation_functions import RELU_Layer
from utils_cnn.activation_functions import RELU_layer_backward
from utils_cnn.inits import he_initialization
from utils_cnn import losses

In [2]:
(X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes) = load_dataset()

In [3]:
X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))

number of training examples = 1080
number of test examples = 120


## Convolutional Layer

In [4]:
np.random.seed(1)
W = he_initialization((3,3,3,8))
b = he_initialization((1,1,1,8))

Z, cache_conv = conv_layer(X_train[:50], W, b, {"pad": 1, "stride": 1})
print("Z's mean =", np.mean(Z))
print("shape: ")
Z.shape

("Z's mean =", 1.4875449904865423)
shape: 


(50, 64, 64, 8)

## MAX Pooling Layer

In [5]:
A, cache_pool = pool_forward(Z, {"stride" : 2, "f": 2}, mode="max")
A.shape

(10, 64)

## Test of the conv backward pass

In [6]:
dA_prev, dW, db = convolutional_backward(Z, cache_conv)
print("size dA_prev: " + str(dA_prev.shape))
print("size dW: " + str(dW.shape))
print("size db: " + str(db.shape))

size dA_prev: (50, 64, 64, 3)
size dW: (3, 3, 3, 8)
size db: (1, 1, 1, 8)


## Test of the maxpool pass

In [7]:
dA_prev = pool_backward(A, cache_pool, mode="max")
print("size dA_prev: " + str(dA_prev.shape))

size dA_prev: (50, 64, 64, 8)


In [27]:
def forward_pass(X):
    W1_shape = [5,5,3,8]
    W2_shape = [3,3,8,16]
    W1 = he_initialization(W1_shape)
    W2 = he_initialization(W2_shape)
    b1 = np.zeros((1,1,1,8))
    b2 = np.zeros((1,1,1,16))
    
    cache = {}
    
    Z1, cache["Z1"] = conv_layer(X, W1, b1, {"pad": 2, "stride": 1})
    A1 = RELU_Layer(Z1)
    P1, cache["P1"] = pool_forward(A1, {"f": 8, "stride": 8})
    Z2, cache["Z2"] = conv_layer(P1, W2, b2, {"pad": 1, "stride": 1})
    A2 = RELU_Layer(Z2)
    P2, cache["P2"] = pool_forward(A2, {"f": 4, "stride": 4})
    m, h, w, c = P2.shape
    P2flat = np.zeros((m, h*w*c))
    
    for i in range(m):
        P2flat[i] = P2[i].flatten()
        
    W3 = np.random.randn(6, P2flat.shape[1]) * 0.01
    b1 = np.zeros((W3.shape[0], 1))
    
    Z3, cache["Z3"] = linear_forward(P2flat.T, W3,b1)
    ce = losses.CrossEntropy()
    Z3 = ce._softmax(Z3.T)

    return Z3, cache

In [ ]:
def backward_pass(X):
    

In [9]:
W1_shape = [5,5,3,8]
W2_shape = [3,3,8,16]
W1 = he_initialization(W1_shape)
W2 = he_initialization(W2_shape)
b1 = he_initialization((1,1,1,8))
b2 = he_initialization((1,1,1,16))

In [28]:
A, cache = forward_pass(X_train[:10])

In [31]:
A[1,0]= 1

In [29]:
ce = losses.CrossEntropy()

In [52]:
A[0].flatten()

array([ 0.        ,  0.        ,  0.45160174,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.01804696,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.45665179,  0.        ,  0.        ,  0.44736449,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.03423493,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.43725551,  0.        ,  0.        ,  0.41592996,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.9649079 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.44027795,  0.        ,  0.        ,
        0.38333921,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.97798408,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.44668971])

In [34]:
cache = {}
    
Z1, cache["Z1"] = conv_layer(X_train[:10], W1, b1, {"pad": 2, "stride": 1})

In [35]:
Z1.shape

(10, 64, 64, 8)

In [37]:
A1 = RELU_Layer(Z1)
A1.shape

(10, 64, 64, 8)

In [38]:
P1, cache["P1"] = pool_forward(A1, {"f": 8, "stride": 8})
P1.shape

(10, 8, 8, 8)

In [39]:
Z2, cache["Z2"] = conv_layer(P1, W2, b2, {"pad": 1, "stride": 1})
Z2.shape

(10, 8, 8, 16)

In [47]:
Z2.flatten().shape

(10240,)